In [1]:
import os
import CityHub
import folium
import geopandas as gpd
import pandas as pd

In [2]:
ch = CityHub.CityHub('data/sp_network.gpickle',40.0)

Loading file...
Preprocessing city mesh...
Refining mesh...


In [3]:
ch.load_PALayer_mesh('data/malha-sp-setores-2010.kml','Name',40.0)

Refining mesh...


0

In [4]:
ch.load_PALayers_csv_data(0,'data/sectorsindices.csv','Cod_setor')

True

In [5]:
ch.load_PBLayer('data/SAD69-96_SHP_pontoonibus.shp',23,'K',True,-45.0,-45.0)

0

In [6]:
ch.load_PBLayer('data/SAD69-96_SHP_pontoonibus.shp',23,'K',True,-45.0,-45.0)

1

In [7]:
ch.load_PBLayer('data/SAD69-96_SHP_estacaometro_point.shp',23,'K')

2

In [8]:
ch.load_PBLayer('data/SAD69-96_SHP_estacaotrem_point.shp',23,'K')

3

In [9]:
ch.load_PBLayer('data/sad6996_terminal_onibus.shp',23,'K')

4

In [10]:
ch.load_RDLayer('data/AGSN_2019.shp')

0

In [11]:
ch.save_preprocessed_CityHub('data/sp-sibgrapi2022.bin')

In [12]:
ch.load_crimes_layer('data/crimes_ssp.csv','LATITUDE','LONGITUDE')

True

In [13]:
ch.compute_layer_crimes(initial_date = pd.to_datetime('01-01-2020'), final_date = pd.to_datetime('31-12-2020'))

C:\Users\tog2006\AppData\Local\Temp\ipykernel_7712\2885865079.py:1: UserWarning: Parsing '31-12-2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  ch.compute_layer_crimes(initial_date = pd.to_datetime('01-01-2020'), final_date = pd.to_datetime('31-12-2020'))


True

In [14]:
ch.layer_crimes_count

,crime_mobile,crime_vehicle,crime_all
0,3.0,1.0,4.0
1,4.0,0.0,4.0
2,0.0,0.0,0.0
3,1.0,0.0,1.0
4,2.0,0.0,2.0
...,...,...,...
119743,0.0,0.0,0.0
119744,0.0,0.0,0.0
119745,3.0,0.0,3.0
119746,1.0,0.0,1.0


In [15]:
### Sparse-Measurement (SM) Layer ###

# Load sparse measurements related to weather data measured at 3 weather stations
ch.load_SMLayers_known_measurements('Barueri','data/Barueri.CSV')
ch.load_SMLayers_known_measurements('Interlagos','data/Interlagos.CSV')
ch.load_SMLayers_known_measurements('Mirante','data/Mirante.CSV')

# Load the temporal aggregation functions and measurement points infos
ch.load_SMLayers_temp_agg_funcs('data/Sparse_temp_agg_funcs.CSV')
ch.load_SMLayers_measurements_points_info('data/Stations_info.CSV')

# Compute the layer on desired time window
ch.compute_layer_sparse('01-01-2020','31-12-2020')

True

In [16]:
ch.SMLayers_estimated_measurements

,Precipitacao_Total,Temperatura_Maxima,Temperatura_Minima
corner,,,
0,1372.100297,37.557652,7.558559
1,1372.519998,37.565831,7.568262
2,1374.435427,37.576437,7.583988
3,1369.868512,37.573657,7.570547
4,1288.175711,37.502308,7.307016
...,...,...,...
119743,1357.441433,37.743963,7.72407
119744,1360.264695,37.749529,7.736491
119745,1360.110244,37.748781,7.735337


In [ ]:
ch.save_preprocessed_CityGraph('data/sp-sibgrapi2022.bin')

In [ ]:
ch.compute_feature_vector(0)

In [ ]:
import folium
import osmnx as ox
import networkx as nx

lat = -23.549090314066356
long = -46.65458954194309

rectangle_center = [lat, long]

result=ch.query_point_in_graph_radius_in_city_mesh(lat,long,0.7)

In [ ]:
m = folium.Map(location=[lat, long], zoom_start=15, tiles='CartoDB positron')

nearest=ch.query_point_in_city_mesh(lat, long, True)
print(nearest)
folium.CircleMarker(rectangle_center, radius=5, color='blue').add_to(m)
folium.CircleMarker(ch.city_vert_list[nearest], radius=8, color='pink').add_to(m)

for vert in result:
    folium.CircleMarker(ch.city_vert_list[vert], radius=5, color='red').add_to(m)  
    
    
rectangle_side = 0.01

for u,v,a in ch.city_mesh_graph.edges(data=True):
    vertA = ch.city_vert_list[ch.city_vert_nxind_to_ind_dict[u]]
    vertB = ch.city_vert_list[ch.city_vert_nxind_to_ind_dict[v]]  
    if ((abs(vertA[0]-rectangle_center[0])<rectangle_side and abs(vertA[1]-rectangle_center[1])<rectangle_side) or
        (abs(vertB[0]-rectangle_center[0])<rectangle_side and abs(vertB[1]-rectangle_center[1])<rectangle_side)):
            folium.PolyLine([vertA,vertB], color="yellow", weight=2.5, opacity=1).add_to(m)
            folium.CircleMarker(vertA, radius=2, color='blue').add_to(m)  
            folium.CircleMarker(vertB, radius=2, color='blue').add_to(m)    

m      